In [1]:
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import textdistance
import os
from datetime import datetime
from datetime import timedelta

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer



# Constants ------------------------------------------------------------------------------
# fed url
DAY_FS='[YYYYMMDD]'
URL_FS= 'https://www.federalreserve.gov/newsevents/pressreleases/monetary[YYYYMMDD]a.htm'
FROM_DATE = datetime(2020, 1, 29, 0, 0, 1)
global lastFSD

# needleman_wunsch width
match = 1
mismatch = 1
gap = 2


# Methods scraping and feed ---------------------------------------------------------------
def nextBusinessDay(d):
    current_date = d
    while True:
        current_date += timedelta(days=1)
        weekday = current_date.weekday()
        if weekday >= 5:
            continue
        else:
            return current_date
def createNPL(sad, url, d): 
    nplBloc = """
                <br>
                <div style="background-color:#FCF3CF;"> 
                    <h3>Sentiment Analysis for Fed Statment date: <a href='[url]'>[date]</h3>
                    <h4><span style="color:#008000;">Positive  : </span>[+]</h4>
                    <h4><span style="color:#FF0000;">Negative  : </span>[-]</h4>
                    <h4><span style="color:#808080;">Neutre    : </span>[=]</h4>
                    <h4><span style="color:#000000;">Compouned : </span>[x]</h4>
                  
                 </div>
              """
    nplBloc = nplBloc.replace('[url]', url)
    nplBloc = nplBloc.replace('[date]', d)
    
    sia = SentimentIntensityAnalyzer()
    pscore = sia.polarity_scores(sad)
    nplBloc = nplBloc.replace('[+]', str(pscore["pos"]))
    nplBloc = nplBloc.replace('[-]', str(pscore["neg"]))
    nplBloc = nplBloc.replace('[=]', str(pscore["neu"]))
    nplBloc = nplBloc.replace('[x]', str(pscore["compound"]))
    
    return nplBloc



def strLoadArticle(strdate):
    datedArticle = 'fedStates/'+ strdate
    if os.path.exists(datedArticle):
        return datedArticle
    else:
        datedUrl = URL_FS.replace(DAY_FS, strdate) 
        try:
            page = requests.get(datedUrl)
            soup = BeautifulSoup(page.content, 'html.parser')
            divs =  soup.find(id="article").findAll('div')
            ldivs = list(map(lambda d: len(d.findChildren("p", recursive=False)), divs))
            articleDiv = divs[ldivs.index(max(ldivs))]
            ps = articleDiv.findAll('p')
            
            if ps[-1].text.startswith('Implementation Note issued') :
                for tag in articleDiv.findAll(True):
                    if len(tag.attrs)> 0 :
                        tag.attrs = {}
                content = list(map(lambda p: ('<p>'+str(p)+'</p>'), ps))
                with open(datedArticle, "w") as f:
                    sad = str("<br>".join(content))
                    sad += createNPL(sad, datedUrl, strdate)
                    f.write(sad)
                return datedArticle 
            else:
                return None
        except:
            return None
        

def loadArticle(d):
    strdate = d.strftime("%Y%m%d")
    return strLoadArticle(strdate)
    
def extractArticle(sdate):
    datedArticle = strLoadArticle(sdate)
    if not datedArticle is None:
        with open(datedArticle, 'r') as f:
             content = f.read()
        return content
    else:
        return ""
        
def splitArticle(a):
    a = a.replace('<p>','')
    a = a.replace('</p>','')
    return a.split('<br>')[:-1]
        
def startupArticles():
    global lastFSD
    dates = []
    dateRegex = re.compile(r'(\d\d\d\d\d\d\d\d)')

    for datedFile in os.listdir('fedStates/'):
        if dateRegex.search(datedFile):
            dates.append(datedFile)
    fromDate = FROM_DATE
    if dates:
        try:
            dates.sort(reverse=True)
            fromDate = datetime.strptime(dates[0], '%Y%m%d')
            lastFSD = dates[0]
        except:
            fromDate = FROM_DATE
    
    dnow = datetime.now()
    while fromDate <= dnow :
        if not loadArticle(fromDate) is None:
            lastFSD = fromDate.strftime("%Y%m%d") 
        fromDate = nextBusinessDay(fromDate)

        
def dailyCheckArticle():
    global lastFSD
    sdnow = datetime.now().strftime("%Y%m%d")
    if lastFSD < sdnow:
        if not strLoadArticle(sdnow) is None:
            lastFSD = sdnow
    return lastFSD

#Methods Diffs Algo -------------------------------------------------------------------------------
def iso(x, y, sCoef = 1):
    if textdistance.cosine(x,y) >= sCoef:
        return True
    else:
        return False

def score(x, y, sCoef = 1):
    if iso(x,y, sCoef):
        return match
    else:
        return -mismatch
     
    
def needleman_wunsch(image, shadow, sCoef = 1):
    m=len(image)
    n=len(shadow)
    F = np.zeros((m  , n ))
    F[:,0] = -np.arange(0, m*gap,gap)
    F[0,:] = -np.arange(0, n*gap,gap)
            
    for i in range(1,m):
        for j in range(1,n):
            s = score(image[i], shadow[j], sCoef)
            F[i][j] = max(F[i-1][j-1] + s, F[i-1][j] - gap, F[i][j-1] - gap)
    return F



def nw_traceback(image , shadow, F, sCoef = 1):
    m=len(F)
    n=len(F[0])
    outImage=[]
    outShadow=[]
    i,j=m-1,n-1
    while i>=0 and j>=0:
        matching = iso(image[i],shadow[j], sCoef)
        if F[i-1][j]==F[i][j-1] or matching:
            if matching:
                outImage.append(image[i])
                outShadow.append(shadow[j])
            else :
                outImage.append(image[i])
                outImage.append("_")
                outShadow.append("_")
                outShadow.append(shadow[j])
            i-=1
            j-=1
            
        elif F[i-1][j]>F[i][j-1]:
            outImage.append(image[i])
            outShadow.append("_")
            i-=1
        else:
            outImage.append("_")
            outShadow.append(shadow[j])
            j-=1
    if i>0:
        while i>=0:
            if image[i]==shadow[j]:
                outImage.append(image[i])
                outShadow.append(shadow[j])
                i-=1
                j-=1
            else:
                outImage.append(image[i])
                outShadow.append("_")
                i-=1
    if j>0:
        while j>=0:
            if shadow[j]==image[i]:
                outImage.append(image[i])
                outShadow.append(shadow[j])
                i-=1
                j-=1
            else:
                outImage.append("_")
                outShadow.append(shadow[j])
                j-=1
    if i==0 and j==0:
        outImage.append(image[i])
        outShadow.append(shadow[j])
    elif i==0:
        outImage.append(image[i])
        outShadow.append("_")
    elif j==0:
        outImage.append("_")
        outShadow.append(shadow[j])

    return outImage,outShadow


def processParagraph(pim, psh):
    if pim is None and psh is None:
        return ""
    if pim is None:
        return '<p><span class="sh">' + psh + '</span></p>'
    if psh is None:
        return '<p><span class="im">' + pim + '</span></p>'
    
    wim = pim.split(' ')
    wsh = psh.split(' ')
    F = needleman_wunsch(wim,wsh)
    outImage,outShadow = nw_traceback(wim, wsh, F)
    outImage = outImage[::-1]
    outShadow = outShadow[::-1]
    binMatch = []
    for i in range(len(outImage)):
        if outImage[i] != '_' and outShadow[i] != '_' :
            binMatch.append(1)
        else:
            binMatch.append(0)
    res = ['<p>']
    image = []
    shadow = []
    bl = len(binMatch)-1
    for i in range(bl+1):
        if binMatch[i] == 0 :
            if i == 0 or (i>0 and binMatch[i-1] == 1):
                image.append('<span class="im">') 
                shadow.append('<span class="sh">') 
            if outImage[i] != '_' :
                image.append(outImage[i])
            if outShadow[i] != '_':
                shadow.append(outShadow[i])
            if i == bl or (i < bl and binMatch[i+1] == 1):
                image.append('</span>') 
                shadow.append('</span>') 
                res.extend(shadow)
                res.extend(image)
                image = []
                shadow = []
        else:
            res.append(outImage[i])
    
    res.append('</p>')
    return ' '.join(res)
     

def processArticle(content1, content2):
    delta = []
    l1 = len(content1)
    l2 = len(content2)
    for i in range(l2):
        if i < l1 :
            delta.append(processParagraph(content2[i], content1[i]))
        else:
            delta.append(processParagraph(content2[i], None))
    if l2 < l1:
        for i in range(l2, l1):
            delta.append(processParagraph(None, content1[i]))
    return '<br>'.join(delta)   

<a>Implementation Note issued July 28, 2021</a>
fedStates/20210728


import nltk
nltk.download()